<a href="https://colab.research.google.com/github/dooooing/2023-ESAA-/blob/main/%EC%8B%A0%EC%9A%A9%EC%B9%B4%EB%93%9C_%EC%BD%94%EB%93%9C_%EA%B3%B5%EC%9C%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LGBM

## 0. LIBRARY 불러오기

In [59]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import random

## 1. 데이터 전처리

In [60]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [61]:
submit = pd.read_csv('sample_submission.csv')

index 칼럼 삭제

In [62]:
train = train.drop(['index'], axis=1)
test = test.drop(['index'], axis=1)

missing value는 'NAN' 문자열로 대체

In [63]:
train.fillna('NAN', inplace=True)
test.fillna('NAN', inplace=True)

object(문자열) onehot encoding

In [64]:
object_col = []
for col in train.columns:
    if train[col].dtype == 'object':
        object_col.append(col)

In [65]:
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(),
             columns=enc.get_feature_names_out(object_col))
train.drop(object_col, axis=1, inplace=True)
train = pd.concat([train, train_onehot_df], axis=1)

In [66]:
test_onehot_df = pd.DataFrame(enc.transform(test.loc[:,object_col]).toarray(),
             columns=enc.get_feature_names_out(object_col))
test.drop(object_col, axis=1, inplace=True)
test = pd.concat([test, test_onehot_df], axis=1)

## 2. 모델링

stratifiedKFold를 사용해 y값 분포를 비슷하게 분리

In [67]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))

In [68]:
random.seed(42)
lgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values
    lgb = LGBMClassifier(n_estimators=1000)
    lgb.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_train, y_train), (X_valid, y_valid)])
    lgb_models[fold]=lgb
    print(f'================================================================================\n\n')

====================================1============================================
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004044 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 813
[LightGBM] [Info] Number of data points in the train set: 21165, number of used features: 53
[LightGBM] [Info] Start training from score -2.105723
[LightGBM] [Info] Start training from score -1.440314
[LightGBM] [Info] Start training from score -0.444119


====================================2============================================
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012250 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 819
[LightGBM] [Info] Number of data points in the train se

In [69]:
submit.iloc[:,1:]=0
for fold in range(5):
    submit.iloc[:,1:] += lgb_models[fold].predict_proba(test)/5

In [70]:
submit.to_csv('submit_lgbm.csv', index=False)

# Randomforest